**This script focuses on the basic concept of finding common words after removing stop words**

Let us try and beat the benchmark

In [ ]:
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from scipy.optimize import minimize

In [ ]:
stops = set(stopwords.words("english"))

In [ ]:
from numpy import prod
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().replace("-","").replace("."," ").rstrip('?').split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().replace("-","").replace("."," ").rstrip('?').split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words = [len(w) for w in q1words.keys() if w in q2words]
    count1 = prod(shared_words)*prod(shared_words)
    shared_prod1 = [len(w) for w in q1words]
    shared_prod2 = [len(w) for w in q2words]
    count2 = prod(shared_prod1)*prod(shared_prod1)
    R = count1/count2
    return R

In [ ]:
train = pd.read_csv("../input/train.csv")
train[ 'R' ] = train.apply( word_match_share, axis=1, raw=True )
print( train.head() ) 

In [ ]:
test = pd.read_csv("../input/test.csv", index_col=False )
test['R'] = test.apply( word_match_share, axis=1, raw=True )
print( test.head() )

**Mean Target**

In [ ]:
GLOBAL_MEAN = np.mean( train['is_duplicate'] ) 
print( 'Mean is_duplicated', GLOBAL_MEAN )

In [ ]:
#OPTIMIZE FUNCTIONS
def minimize_train_log_loss( W ):
    train["prediction"] = GLOBAL_MEAN + train["R"] * W[0] + W[1]
    score = log_loss( train['is_duplicate'], train['prediction'] )
    print(  score , W )
    return( score )

**OPTIMIZE FUNCTIONS**

In [ ]:
res = minimize(minimize_train_log_loss, [0.00,  0.00], method='Nelder-Mead', tol=1e-4, options={'maxiter': 400})
W = res.x
print( 'Best weights: ',W )

**APPLY TO TESTSET**

In [ ]:
test["is_duplicate"] = GLOBAL_MEAN + test["R"] * W[0] + W[1]
test[ ['test_id','is_duplicate'] ].to_csv("count_words_benchmark.csv", header=True, index=False)	